In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import numpy as np
import cv2

rootPath = "D:/DTU-D/4/Fagprojekt/Data"
csvName = "labels.csv"
csvName2 = "labels2.csv"
data = pd.concat([
    pd.read_csv(os.path.join(rootPath,csvName)),
    pd.read_csv(os.path.join(rootPath,csvName2))
])
# data.drop(data.index[[1711]], inplace=True) #GIF
data.drop(data.index[[3033,3034,3035]], inplace=True) #Corrupted pictures
rows = data.shape[0]

In [2]:
# Get picture resolutions
resolutions = np.zeros(shape=(rows,2))
for i in tqdm(range(rows)):
    image_path = data.iloc[i]['image_path'].lower()
    if image_path.endswith("gif") or image_path.endswith("pdf"):
        continue
    # The replaced letters are NOT the same. There is a mismatch between the csv file and the actual picture name
    if "ä" in image_path:
        image_path = image_path.replace("ä", "ä")
    if "å" in image_path:
        image_path = image_path.replace("å", "å")
    if "ö" in image_path:
        image_path = image_path.replace("ö", "ö")

    path = os.path.join(rootPath,image_path).lower()
    # imdecode is needed to read paths with letters øæå
    # https://stackoverflow.com/questions/43185605/how-do-i-read-an-image-from-a-path-with-unicode-characters
    img = cv2.imdecode(np.fromfile(path, np.uint8), cv2.IMREAD_UNCHANGED)
    resolutions[i] = [img.shape[0],img.shape[1]]

  0%|          | 0/3495 [00:00<?, ?it/s]

In [3]:
# Get unique resolutions
unique_reso = []
unique_reso.append(resolutions[0])
for i in range(len(resolutions)):
    unique = True
    for j in range(len(unique_reso)):
        if resolutions[i][0] == unique_reso[j][0] and resolutions[i][1] == unique_reso[j][1]:
            unique = False
            break
    if unique:
        unique_reso.append(resolutions[i])

In [4]:
# Get count on unique resolutions
def getCount(array, toLook):
    count = 0
    for i in range(len(toLook)):
        if (array==toLook[i]).all():
            count += 1
    return count

dict_unique = []
for i in range(len(unique_reso)):
    dict_unique.append((unique_reso[i],getCount(unique_reso[i], resolutions)))
reso_count = np.array(dict_unique)

C:\Users\Damho\AppData\Local\Temp/ipykernel_126408/2467117560.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  reso_count = np.array(dict_unique)


In [5]:
print(f"Number of pictures: {rows}")
print(f"number of unique resolutions: {len(reso_count)}")
print(f"Number of resolutions with more than 5 pictures: {np.sum(reso_count[:,1]>5)}")
print(f"Number of resolutions with more than 10 pictures: {np.sum(reso_count[:,1]>10)}")
print(f"Number of resolutions with more than 15 pictures: {np.sum(reso_count[:,1]>15)}")
print(f"Number of resolutions with more than 20 pictures: {np.sum(reso_count[:,1]>20)}")
print(f"Number of resolutions with more than 40 pictures: {np.sum(reso_count[:,1]>40)}")

Number of pictures: 3495
number of unique resolutions: 289
Number of resolutions with more than 5 pictures: 46
Number of resolutions with more than 10 pictures: 32
Number of resolutions with more than 15 pictures: 22
Number of resolutions with more than 20 pictures: 16
Number of resolutions with more than 40 pictures: 12
